# Introduction
This file extracts information from the Bureau of Labor Statistics regarding average salaries and total workers in Whitman County, WA between 1990 and 2022. 

### Output
* 'data512_common_analysis/data/wages_timeseries.csv'
    > Average wage in dollars for ambulatory health care service workers between 2001 and 2022.
* 'data512_common_analysis/data/employee_timeseries.csv' 
    > Average number of ambulatory health care service workers in thousands for every month in the fire season between 1990 and 2023. 

### Acknowledgment
This is adapted from the work of Mark McEnearney whose source code is provided as reference by the BLS [here](https://www.bls.gov/developers/api_python.htm). The contents of this file use the [BLS API](https://www.bls.gov/developers/api_signature_v2.htm) (see [Terms of Service](https://www.bls.gov/developers/termsOfService.htm)) BLS.gov cannot vouch for the data or analyses derived from these data after the data have been retrieved from BLS.gov.

### License
This code example was developed by Chandler Ault for use in DATA 512, a course in the UW MS Data Science degree program. This code is provided under the [Creative Commons](https://creativecommons.org) [CC-BY license](https://creativecommons.org/licenses/by/4.0/). 


In [1]:
import requests
import json
import pandas as pd

In [ ]:

import requests
import json
import prettytable
import pandas as pd
headers = {'Content-type': 'application/json'}

s = ['ENU53075305621','CUSR0000SA0'] # If the first code does not retrieve data, go to https://www.bls.gov/data/ to retrieve wage data.

cpi= []
cpi_month = []
cpi_year = []
wages = []
wage_year = []
for i in range(2000, 2023,10):

  data = json.dumps({"seriesid": s,"startyear":str(i), "endyear":str(i+9)})
  p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)

  json_data = json.loads(p.text)

  fire_season = ['May', 'June', 'July', 'August', 'September', 'October']
  for series in json_data['Results']['series']:
        seriesId = series['seriesID']
        if seriesId == 'CUSR0000SA0':
            for item in series['data']:
                if item['periodName'] in fire_season:
                    cpi.append(float(item['value']))
                    cpi_month.append(item['periodName'])
                    cpi_year.append(int(item['year']))
        elif seriesId == 'ENU53075505621':
            for item in series['data']:
                wages.append(float(item['value']))
                wage_year.append(int(item['year']))
        else:
            pass

cpi_data = list(zip(cpi, cpi_month, cpi_year))
df_cpi = pd.DataFrame(cpi_data, columns=['cpi', 'Month', 'Year'])
df_cpi.to_csv('cpi.csv')

wage_data = list(zip(wages, wage_year))
df = pd.DataFrame(wage_data, columns=['Wages', 'Year'])
df.to_csv('wages_timeseries.csv')

